In [1]:
%reset -fs
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pandas_profiling as pp
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
df = pd.read_csv('fuzzy_matches_bet_df.csv', index_col=0)
df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,pick_over_under,seven_day_accuracy,successful_pick,Matches
0,2021-10-21,Bogdan,ATL,G,DAL,vs.,15.5,-115,16.57,1.07,16.0,11,o,0.47,0,"[('Bogdan Bogdanović', 90), ('Bogdan Bogdanovi..."
1,2021-10-23,Bogdan,ATL,G,CLE,@,14.5,-115,15.29,0.79,14.5,5,o,0.45,0,"[('Bogdan Bogdanović', 90), ('Bogdan Bogdanovi..."
2,2021-10-25,Bogdan,ATL,G,DET,vs.,15.5,-115,12.85,-2.65,13.5,14,u,0.45,0,"[('Bogdan Bogdanović', 90), ('Bogdan Bogdanovi..."
3,2021-10-28,Bogdan,ATL,G,WAS,@,12.5,-105,16.72,4.22,12.5,14,o,0.42,1,"[('Bogdan Bogdanović', 90), ('Bogdan Bogdanovi..."
4,2021-10-30,Bogdan,ATL,G,PHI,@,12.5,-113,15.71,3.21,12.5,13,o,0.45,1,"[('Bogdan Bogdanović', 90), ('Bogdan Bogdanovi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4621,2021-12-04,Zeke Nnaji,DEN,F,NYK,@,5.5,-105,2.91,-2.59,5.5,21,u,0.55,0,"[('Zeke Nnaji', 100), ('Zeke Nnaji', 100)]"
4622,2021-12-06,Zeke Nnaji,DEN,F,CHI,@,7.5,115,3.35,-4.15,7.5,3,u,0.51,1,"[('Zeke Nnaji', 100), ('Zeke Nnaji', 100)]"
4623,2021-10-28,Ziaire Williams,MEM,G,GSW,@,5.5,-130,4.33,-1.17,5.5,4,u,0.42,1,"[('Ziaire Williams', 100), ('Ziaire Williams',..."
4624,2022-01-17,Ziaire Williams,MEM,G,CHI,vs.,8.5,-105,6.11,-2.39,8.5,2,u,0.58,1,"[('Ziaire Williams', 100), ('Ziaire Williams',..."


In [3]:
df2 = pd.read_csv('player_team_stats_merged.csv', index_col=0)
df2

,Player,Age,Pos,Tm,Home/Away_x,Opp,Game Result,Date,GS,MP_x,FG_x,FGA_x,FG%_x,2P_x,2PA_x,2P%_x,3P_x,3PA_x,3P%_x,FT_x,FTA_x,FT%_x,TS%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS_x,GmSc,BPM,Home/Away_y,Result,MP_y,FG_y,FGA_y,FG%_y,2P_y,2PA_y,2P%_y,3P_y,3PA_y,3P%_y,FT_y,FTA_y,FT%_y,PTS_y,FG.1,FGA.1,FG%.1,2P.1,2PA.1,2P%.1,3P.1,3PA.1,3P%.1,FT.1,FTA.1,FT%.1,PTS.1
0,Aaron Gordon,26-034,F,DEN,Away,PHO,W,2021-10-20,1,29,5,6,0.833,4,5,0.800,1,1,1.000,1,2,0.50,0.872,2,6,8,4,0,1,3,2,12,12.3,6.1,Away,W 110-98,240,44,83,0.530,27,44,0.614,17,39,0.436,5,9,0.556,110,36,87,0.414,22,50,0.440,14,37,0.378,12,17,0.706,98
1,Aaron Gordon,26-036,F,DEN,Home,SAS,W,2021-10-22,1,30,3,6,0.500,3,5,0.600,0,1,0.000,2,2,1.00,0.581,2,3,5,1,0,0,3,5,8,3.0,-8.1,Home,W 102-96,240,44,86,0.512,35,56,0.625,9,30,0.300,5,8,0.625,102,40,84,0.476,34,60,0.567,6,24,0.250,10,14,0.714,96
2,Aaron Gordon,26-039,F,DEN,Home,CLE,L,2021-10-25,1,28,5,14,0.357,4,9,0.444,1,5,0.200,1,2,0.50,0.403,3,6,9,1,0,0,3,2,12,4.6,-6.5,Home,L 87-99,240,35,86,0.407,26,48,0.542,9,38,0.237,8,12,0.667,87,38,82,0.463,31,57,0.544,7,25,0.280,16,20,0.800,99
3,Aaron Gordon,26-040,F,DEN,Away,UTA,L,2021-10-26,1,34,9,13,0.692,9,10,0.900,0,3,0.000,2,4,0.50,0.678,1,6,7,1,1,1,4,5,20,12.6,-3.2,Away,L 110-122,240,42,83,0.506,32,55,0.582,10,28,0.357,16,24,0.667,110,44,85,0.518,32,46,0.696,12,39,0.308,22,29,0.759,122
4,Aaron Gordon,26-043,F,DEN,Home,DAL,W,2021-10-29,1,24,6,9,0.667,5,7,0.714,1,2,0.500,0,1,0.00,0.689,1,4,5,2,1,1,2,1,13,11.3,8.3,Home,W 106-75,240,43,83,0.518,32,47,0.681,11,36,0.306,9,13,0.692,106,23,78,0.295,15,41,0.366,8,37,0.216,21,27,0.778,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14997,Ziaire Williams,20-127,F,MEM,Home,CHI,W,2022-01-17,1,18,0,4,0.000,0,1,0.000,0,3,0.000,2,2,1.00,0.205,0,3,3,1,0,0,2,2,2,-2.0,-17.9,Home,W 119-106,240,47,92,0.511,37,67,0.552,10,25,0.400,15,20,0.750,119,40,92,0.435,27,61,0.443,13,31,0.419,13,15,0.867,106
14998,Ziaire Williams,20-129,F,MEM,Away,MIL,L,2022-01-19,1,33,4,11,0.364,3,4,0.750,1,7,0.143,1,1,1.00,0.437,0,1,1,1,1,0,0,1,10,5.5,-6.3,Away,L 114-126,240,44,106,0.415,35,66,0.530,9,40,0.225,17,25,0.680,114,46,97,0.474,33,60,0.550,13,37,0.351,21,22,0.955,126
14999,Ziaire Williams,20-131,F,MEM,Away,DEN,W,2022-01-21,1,31,6,8,0.750,4,5,0.800,2,3,0.667,3,4,0.75,0.871,1,4,5,0,2,1,0,4,17,16.4,8.1,Away,W 122-118,240,43,82,0.524,32,57,0.561,11,25,0.440,25,40,0.625,122,43,91,0.473,30,59,0.508,13,32,0.406,19,23,0.826,118
15000,Ziaire Williams,20-133,F,MEM,Away,DAL,L,2022-01-23,1,23,4,13,0.308,3,5,0.600,1,8,0.125,0,0,NaN,0.346,1,0,1,1,0,0,0,5,9,0.9,-9.6,Away,L 91-104,240,37,99,0.374,33,75,0.440,4,24,0.167,13,19,0.684,91,36,74,0.486,27,44,0.614,9,30,0.300,23,31,0.742,104


In [4]:
matches = df['Matches'].str.split(',', expand=True)
matches

,0,1,2,3
0,[('Bogdan Bogdanović',90),('Bogdan Bogdanović',90)]
1,[('Bogdan Bogdanović',90),('Bogdan Bogdanović',90)]
2,[('Bogdan Bogdanović',90),('Bogdan Bogdanović',90)]
3,[('Bogdan Bogdanović',90),('Bogdan Bogdanović',90)]
4,[('Bogdan Bogdanović',90),('Bogdan Bogdanović',90)]
...,...,...,...,...
4621,[('Zeke Nnaji',100),('Zeke Nnaji',100)]
4622,[('Zeke Nnaji',100),('Zeke Nnaji',100)]
4623,[('Ziaire Williams',100),('Ziaire Williams',100)]
4624,[('Ziaire Williams',100),('Ziaire Williams',100)]


In [5]:
matches[0] = matches[0].map(lambda x: x.replace('[','').replace('(', '').replace("'", ''))
matches

,0,1,2,3
0,Bogdan Bogdanović,90),('Bogdan Bogdanović',90)]
1,Bogdan Bogdanović,90),('Bogdan Bogdanović',90)]
2,Bogdan Bogdanović,90),('Bogdan Bogdanović',90)]
3,Bogdan Bogdanović,90),('Bogdan Bogdanović',90)]
4,Bogdan Bogdanović,90),('Bogdan Bogdanović',90)]
...,...,...,...,...
4621,Zeke Nnaji,100),('Zeke Nnaji',100)]
4622,Zeke Nnaji,100),('Zeke Nnaji',100)]
4623,Ziaire Williams,100),('Ziaire Williams',100)]
4624,Ziaire Williams,100),('Ziaire Williams',100)]


In [6]:
df['Matches'] = matches[0]
df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,pick_over_under,seven_day_accuracy,successful_pick,Matches
0,2021-10-21,Bogdan,ATL,G,DAL,vs.,15.5,-115,16.57,1.07,16.0,11,o,0.47,0,Bogdan Bogdanović
1,2021-10-23,Bogdan,ATL,G,CLE,@,14.5,-115,15.29,0.79,14.5,5,o,0.45,0,Bogdan Bogdanović
2,2021-10-25,Bogdan,ATL,G,DET,vs.,15.5,-115,12.85,-2.65,13.5,14,u,0.45,0,Bogdan Bogdanović
3,2021-10-28,Bogdan,ATL,G,WAS,@,12.5,-105,16.72,4.22,12.5,14,o,0.42,1,Bogdan Bogdanović
4,2021-10-30,Bogdan,ATL,G,PHI,@,12.5,-113,15.71,3.21,12.5,13,o,0.45,1,Bogdan Bogdanović
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4621,2021-12-04,Zeke Nnaji,DEN,F,NYK,@,5.5,-105,2.91,-2.59,5.5,21,u,0.55,0,Zeke Nnaji
4622,2021-12-06,Zeke Nnaji,DEN,F,CHI,@,7.5,115,3.35,-4.15,7.5,3,u,0.51,1,Zeke Nnaji
4623,2021-10-28,Ziaire Williams,MEM,G,GSW,@,5.5,-130,4.33,-1.17,5.5,4,u,0.42,1,Ziaire Williams
4624,2022-01-17,Ziaire Williams,MEM,G,CHI,vs.,8.5,-105,6.11,-2.39,8.5,2,u,0.58,1,Ziaire Williams


In [20]:
df[df.Date == '2021-11-03']

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,pick_over_under,seven_day_accuracy,successful_pick,Matches
6,2021-11-03,Bogdan,ATL,G,BKN,@,13.5,-107,16.81,3.31,13.5,8,o,0.55,0,Bogdan Bogdanović
241,2021-11-03,Ayo Dosunmu,CHI,G,PHI,@,6.5,105,4.32,-2.18,6.5,3,u,0.55,1,Ayo Dosunmu
275,2021-11-03,Blake Griffin,BKN,F,ATL,vs.,7.5,-105,11.46,3.96,7.5,7,o,0.55,0,Blake Griffin
340,2021-11-03,Bradley Beal,WAS,G,TOR,vs.,27.5,-110,32.66,5.17,27.5,25,o,0.55,0,Bradley Beal
528,2021-11-03,Caris LeVert,IND,G,NYK,vs.,16.5,-120,13.90,-2.60,16.5,21,u,0.55,0,Caris LeVert
565,2021-11-03,Cedi Osman,CLE,F,POR,vs.,10.5,-110,8.16,-2.34,10.5,12,u,0.55,0,Cedi Osman
586,2021-11-03,Chris Boucher,TOR,F,WAS,@,7.5,-115,10.40,2.90,7.5,5,o,0.55,0,Chris Boucher
607,2021-11-03,Chris Duarte,IND,G,NYK,vs.,14.5,-107,17.59,3.09,14.5,10,o,0.55,0,Chris Duarte
691,2021-11-03,Clint Capela,ATL,C,BKN,@,10.5,-125,15.15,4.65,12.5,13,o,0.55,1,Clint Capela
728,2021-11-03,Cody Martin,CHA,F,GSW,@,6.5,115,10.04,3.54,6.5,2,o,0.55,0,Cody Martin


In [18]:
df['Date'] = pd.to_datetime(df.Date)

In [19]:
df2['Date'] = pd.to_datetime(df2.Date)

In [21]:
df2[(df2.Date=='2021-11-03') & (df2.Tm == 'ATL')]

,Player,Age,Pos,Tm,Home/Away_x,Opp,Game Result,Date,GS,MP_x,FG_x,FGA_x,FG%_x,2P_x,2PA_x,2P%_x,3P_x,3PA_x,3P%_x,FT_x,FTA_x,FT%_x,TS%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS_x,GmSc,BPM,Home/Away_y,Result,MP_y,FG_y,FGA_y,FG%_y,2P_y,2PA_y,2P%_y,3P_y,3PA_y,3P%_y,FT_y,FTA_y,FT%_y,PTS_y,FG.1,FGA.1,FG%.1,2P.1,2PA.1,2P%.1,3P.1,3PA.1,3P%.1,FT.1,FTA.1,FT%.1,PTS.1
1041,Bogdan Bogdanović,29-077,G,ATL,Away,BRK,L,2021-11-03,1,29,3,10,0.300,2,5,0.400,1,5,0.200,1,1,1.000,0.383,0,4,4,1,3,0,3,2,8,3.3,-8.1,Away,L 108-117,240,41,94,0.436,28,59,0.475,13,35,0.371,13,15,0.867,108,43,88,0.489,21,40,0.525,22,48,0.458,9,12,0.75,117
1618,Cam Reddish,22-063,F,ATL,Away,BRK,L,2021-11-03,0,19,1,6,0.167,1,3,0.333,0,3,0.000,0,0,NaN,0.167,0,1,1,2,0,0,1,1,2,-1.5,-15.8,Away,L 108-117,240,41,94,0.436,28,59,0.475,13,35,0.371,13,15,0.867,108,43,88,0.489,21,40,0.525,22,48,0.458,9,12,0.75,117
2221,Clint Capela,27-169,C,ATL,Away,BRK,L,2021-11-03,1,33,5,8,0.625,5,8,0.625,0,0,NaN,3,3,1.000,0.697,4,12,16,1,1,0,0,2,13,16.7,3.4,Away,L 108-117,240,41,94,0.436,28,59,0.475,13,35,0.371,13,15,0.867,108,43,88,0.489,21,40,0.525,22,48,0.458,9,12,0.75,117
2768,Danilo Gallinari,33-087,F,ATL,Away,BRK,L,2021-11-03,0,16,3,7,0.429,2,3,0.667,1,4,0.250,0,0,NaN,0.500,1,2,3,1,0,0,0,0,7,5.3,-1.0,Away,L 108-117,240,41,94,0.436,28,59,0.475,13,35,0.371,13,15,0.867,108,43,88,0.489,21,40,0.525,22,48,0.458,9,12,0.75,117
3115,De'Andre Hunter,23-336,F,ATL,Away,BRK,L,2021-11-03,1,33,10,11,0.909,4,5,0.800,6,6,1.000,0,1,0.000,1.136,1,1,2,0,1,1,4,4,26,19.0,10.8,Away,L 108-117,240,41,94,0.436,28,59,0.475,13,35,0.371,13,15,0.867,108,43,88,0.489,21,40,0.525,22,48,0.458,9,12,0.75,117
5437,Gorgui Dieng,31-289,C,ATL,Away,BRK,L,2021-11-03,0,3,0,1,0.000,0,0,NaN,0,1,0.000,0,0,NaN,0.000,0,1,1,0,0,0,1,1,0,-1.8,-37.5,Away,L 108-117,240,41,94,0.436,28,59,0.475,13,35,0.371,13,15,0.867,108,43,88,0.489,21,40,0.525,22,48,0.458,9,12,0.75,117
7474,John Collins,24-041,F,ATL,Away,BRK,L,2021-11-03,1,37,5,14,0.357,4,9,0.444,1,5,0.200,2,2,1.000,0.437,3,9,12,2,1,2,1,3,13,11.6,-2.7,Away,L 108-117,240,41,94,0.436,28,59,0.475,13,35,0.371,13,15,0.867,108,43,88,0.489,21,40,0.525,22,48,0.458,9,12,0.75,117
8905,Kevin Huerter,23-068,G,ATL,Away,BRK,L,2021-11-03,0,23,7,9,0.778,5,7,0.714,2,2,1.000,0,0,NaN,0.889,0,1,1,1,1,1,2,1,16,12.8,8.5,Away,L 108-117,240,41,94,0.436,28,59,0.475,13,35,0.371,13,15,0.867,108,43,88,0.489,21,40,0.525,22,48,0.458,9,12,0.75,117
9797,Lou Williams,35-007,G,ATL,Away,BRK,L,2021-11-03,0,11,1,6,0.167,1,3,0.333,0,3,0.000,0,0,NaN,0.167,0,3,3,5,1,0,0,1,2,3.2,-1.5,Away,L 108-117,240,41,94,0.436,28,59,0.475,13,35,0.371,13,15,0.867,108,43,88,0.489,21,40,0.525,22,48,0.458,9,12,0.75,117
14022,Trae Young,23-045,G,ATL,Away,BRK,L,2021-11-03,1,37,6,22,0.273,4,16,0.250,2,6,0.333,7,8,0.875,0.411,3,6,9,10,0,0,2,1,21,16.1,-1.5,Away,L 108-117,240,41,94,0.436,28,59,0.475,13,35,0.371,13,15,0.867,108,43,88,0.489,21,40,0.525,22,48,0.458,9,12,0.75,117


In [8]:
df.isna().sum()

Date                  0
Name                  0
Team                  0
Position              0
Opp                   0
Home/Away             0
Line                  0
Moneyline             0
Projection            0
Diff                  0
Pick                  0
Result                0
pick_over_under       0
seven_day_accuracy    0
successful_pick       0
Matches               0
dtype: int64

In [22]:
merged_df = pd.merge(df, df2,  how='left', left_on=['Date','Matches','Team', 'Opp'], right_on = ['Date','Player','Tm', 'Opp'])
merged_df


,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result_x,pick_over_under,seven_day_accuracy,successful_pick,Matches,Player,Age,Pos,Tm,Home/Away_x,Game Result,GS,MP_x,FG_x,FGA_x,FG%_x,2P_x,2PA_x,2P%_x,3P_x,3PA_x,3P%_x,FT_x,FTA_x,FT%_x,TS%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS_x,GmSc,BPM,Home/Away_y,Result_y,MP_y,FG_y,FGA_y,FG%_y,2P_y,2PA_y,2P%_y,3P_y,3PA_y,3P%_y,FT_y,FTA_y,FT%_y,PTS_y,FG.1,FGA.1,FG%.1,2P.1,2PA.1,2P%.1,3P.1,3PA.1,3P%.1,FT.1,FTA.1,FT%.1,PTS.1
0,2021-10-21,Bogdan,ATL,G,DAL,vs.,15.5,-115,16.57,1.07,16.0,11,o,0.47,0,Bogdan Bogdanović,Bogdan Bogdanović,29-064,G,ATL,Home,W,1.0,26.0,4.0,8.0,0.500,1.0,3.0,0.333,3.0,5.0,0.600,0.0,0.0,NaN,0.688,0.0,3.0,3.0,2.0,2.0,0.0,2.0,3.0,11.0,8.1,2.8,Home,W 113-87,240.0,45.0,94.0,0.479,30.0,59.0,0.508,15.0,35.0,0.429,8.0,9.0,0.889,113.0,31.0,93.0,0.333,18.0,50.0,0.360,13.0,43.0,0.302,12.0,13.0,0.923,87.0
1,2021-10-23,Bogdan,ATL,G,CLE,@,14.5,-115,15.29,0.79,14.5,5,o,0.45,0,Bogdan Bogdanović,Bogdan Bogdanović,29-066,G,ATL,Away,L,1.0,27.0,2.0,7.0,0.286,1.0,4.0,0.250,1.0,3.0,0.333,0.0,0.0,NaN,0.357,0.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,5.0,2.1,-5.6,Away,L 95-101,240.0,38.0,99.0,0.384,28.0,65.0,0.431,10.0,34.0,0.294,9.0,15.0,0.600,95.0,37.0,89.0,0.416,30.0,61.0,0.492,7.0,28.0,0.250,20.0,27.0,0.741,101.0
2,2021-10-25,Bogdan,ATL,G,DET,vs.,15.5,-115,12.85,-2.65,13.5,14,u,0.45,0,Bogdan Bogdanović,Bogdan Bogdanović,29-068,G,ATL,Home,W,1.0,25.0,5.0,9.0,0.556,3.0,3.0,1.000,2.0,6.0,0.333,2.0,2.0,1.0,0.709,1.0,2.0,3.0,1.0,0.0,0.0,0.0,1.0,14.0,11.3,1.9,Home,W 122-104,240.0,46.0,90.0,0.511,34.0,58.0,0.586,12.0,32.0,0.375,18.0,21.0,0.857,122.0,40.0,91.0,0.440,31.0,58.0,0.534,9.0,33.0,0.273,15.0,18.0,0.833,104.0
3,2021-10-28,Bogdan,ATL,G,WAS,@,12.5,-105,16.72,4.22,12.5,14,o,0.42,1,Bogdan Bogdanović,Bogdan Bogdanović,29-071,G,ATL,Away,L,1.0,34.0,6.0,11.0,0.545,5.0,7.0,0.714,1.0,4.0,0.250,1.0,1.0,1.0,0.612,0.0,6.0,6.0,4.0,1.0,0.0,0.0,5.0,14.0,12.3,0.1,Away,L 111-122,240.0,48.0,88.0,0.545,42.0,67.0,0.627,6.0,21.0,0.286,9.0,14.0,0.643,111.0,46.0,99.0,0.465,33.0,65.0,0.508,13.0,34.0,0.382,17.0,21.0,0.810,122.0
4,2021-10-30,Bogdan,ATL,G,PHI,@,12.5,-113,15.71,3.21,12.5,13,o,0.45,1,Bogdan Bogdanović,Bogdan Bogdanović,29-073,G,ATL,Away,L,1.0,29.0,6.0,11.0,0.545,5.0,10.0,0.500,1.0,1.0,1.000,0.0,0.0,NaN,0.591,2.0,2.0,4.0,1.0,2.0,0.0,3.0,1.0,13.0,9.0,0.5,Away,L 94-122,240.0,36.0,95.0,0.379,28.0,73.0,0.384,8.0,22.0,0.364,14.0,19.0,0.737,94.0,46.0,86.0,0.535,34.0,48.0,0.708,12.0,38.0,0.316,18.0,19.0,0.947,122.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4621,2021-12-04,Zeke Nnaji,DEN,F,NYK,@,5.5,-105,2.91,-2.59,5.5,21,u,0.55,0,Zeke Nnaji,Zeke Nnaji,20-329,F,DEN,Away,W,0.0,34.0,7.0,13.0,0.538,2.0,4.0,0.500,5.0,9.0,0.556,2.0,2.0,1.0,0.756,2.0,6.0,8.0,0.0,1.0,0.0,0.0,5.0,21.0,16.9,2.8,Away,W 113-99,240.0,44.0,84.0,0.524,24.0,41.0,0.585,20.0,43.0,0.465,5.0,5.0,1.000,113.0,35.0,88.0,0.398,23.0,48.0,0.479,12.0,40.0,0.300,17.0,21.0,0.810,99.0
4622,2021-12-06,Zeke Nnaji,DEN,F,CHI,@,7.5,115,3.35,-4.15,7.5,3,u,0.51,1,Zeke Nnaji,Zeke Nnaji,20-331,F,DEN,Away,L,0.0,15.0,1.0,4.0,0.250,1.0,3.0,0.333,0.0,1.0,0.000,1.0,2.0,0.5,0.307,2.0,1.0,3.0,0.0,0.0,0.0,0.0,1.0,3.0,1.5,-11.4,Away,L 97-109,240.0,38.0,96.0,0.396,27.0,64.0,0.422,11.0,32.0,0.344,10.0,15.0,0.667,97.0,41.0,91.0,0.451,28.0,56.0,0.500,13.0,35.0,0.371,14.0,20.0,0.700,109.0
4623,2021-10-28,Ziaire Williams,MEM,G,GSW,@,5.5,-130,4.33,-1.17,5.5,4,u,0.42,1,Ziaire Williams,Ziaire Williams,20-046,F,MEM,Away,W,0.0,21.0,2.0,9.0,0.222,2.0,2.0,1.000,0.0,7.0,0.000,0.0,0.0,NaN,0.222,0.0,1.0,1.0,1.0,1.0,2.0,0.0,2.0,4.0,1.1,-6.9,Away,W 104-101 (OT),265.0,39.0,97.0,0.402,27.0,49.0,0.551,12.0,48.0,0.250,14.0,14.0,1.000,104.0,39.0,90.0,0.433,25.0,50.0,0.500,14.0,40.0,0.350,9.0,12.0,0.750,101.0
4624,2022-01-1

In [11]:
df[(df.Name == 'Bogdan') & (df.Date == '2021-11-03')]

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,pick_over_under,seven_day_accuracy,successful_pick,Matches


In [23]:
merged_df[merged_df.Player.isna()]

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result_x,pick_over_under,seven_day_accuracy,successful_pick,Matches,Player,Age,Pos,Tm,Home/Away_x,Game Result,GS,MP_x,FG_x,FGA_x,FG%_x,2P_x,2PA_x,2P%_x,3P_x,3PA_x,3P%_x,FT_x,FTA_x,FT%_x,TS%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS_x,GmSc,BPM,Home/Away_y,Result_y,MP_y,FG_y,FGA_y,FG%_y,2P_y,2PA_y,2P%_y,3P_y,3PA_y,3P%_y,FT_y,FTA_y,FT%_y,PTS_y,FG.1,FGA.1,FG%.1,2P.1,2PA.1,2P%.1,3P.1,3PA.1,3P%.1,FT.1,FTA.1,FT%.1,PTS.1
6,2021-11-03,Bogdan,ATL,G,BKN,@,13.5,-107,16.81,3.31,13.5,8,o,0.55,0,Bogdan Bogdanović,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-11-04,Bogdan,ATL,G,UTH,vs.,12.5,-115,15.37,2.87,12.5,8,o,0.53,0,Bogdan Bogdanović,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,2021-11-20,Bogdan,ATL,G,CHA,vs.,12.5,-122,16.82,4.32,12.5,7,o,0.56,0,Bogdan Bogdanović,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,2021-10-26,Aaron Gordon,DEN,F,UTH,@,10.5,-127,12.47,1.98,10.5,20,o,0.43,1,Aaron Gordon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,2022-01-16,Aaron Gordon,DEN,F,UTH,vs.,12.5,-105,11.80,-0.70,12.5,20,u,0.57,0,Aaron Gordon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,2021-10-22,Zach LaVine,CHI,G,NOR,vs.,25.5,-113,28.61,3.11,25.5,32,o,0.47,1,Zach LaVine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4599,2021-10-30,Zach LaVine,CHI,G,UTH,vs.,24.5,-115,27.58,3.08,24.5,26,o,0.45,1,Zach LaVine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4602,2021-11-08,Zach LaVine,CHI,G,BKN,vs.,25.5,-118,28.11,2.60,25.5,24,o,0.44,0,Zach LaVine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4609,2021-11-29,Zach LaVine,CHI,G,CHA,vs.,24.5,-121,28.85,4.35,24.5,25,o,0.49,1,Zach LaVine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
